# TEA-phot
EDIT 02/06/2023

Abby Burrows

UPDATES:

    - allow selection of multiple comparison stars
    
    - perform aperture photometry on multiple comparison stars and read out to file
    

NOTES:
    - not well tested for user input file (versus clicking screen to select targets+comps)
------------

PROGRAMME NAME:
    TEA-Phot.py

    Copyright (C) 2019
    D. M. Bowman (IvS, KU Leuven, Belgium)
    D. L. Holdsworth (JHI, UCLan, UK)

    This program comes with ABSOLUTELY NO WARRANTY.
    
   This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but without any warranty; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

DESCRIPTION:
    - Extract a differential light curve using adaptive elliptical apertures
        for FITS files.
        
    - Source extraction and aperature photometry is performed using the python
        SEP module: https://sep.readthedocs.io/en/v1.0.x/
        
    - The code is optimised for SHOC@SAAO photometry, but can and should be
        easily modified to work with other CCD instruments and telescopes.
        
    - BE AWARE: different instruments have different FITS headers and this
        requires a modification of the code.
        
    - This code requires user input at the command line and clicking on a plot,
        which is preceded by ">>>" in the terminal.
        
    - Warnings and errors are output to the terminal and are preceded by
        '! WARNING:' and '!!! ERROR:', respectively.

USAGE:

    - For help, type at the command line: python TEA-Phot.py --help
    
    - MINIMUM USAGE:
    
        python TEA-Phot.py <observatory> <instrument> <image.fits>
    
WARNINGS WHILST RUNNING (SUPPRESSED):   

    - WARNING: VerifyWarning: Card is too long, comment will be truncated.
        [astropy.io.fits.card]

KNOWN ISSUES/BUGS:

    - programme crahses if multiple (red) sources are within pixel limit of
        user's choice. This usually only occurs if the sources are blurred and
        the resultant PSFs are highly elliptical such that sep.extract() finds
        multiple ellipses at a single star's location
        
    - if user closes a window, programme will hang and becomes difficult to
        kill from the terminal
        
    - sep.sum_ellipse() returns ZeroDivisionError if annuli option is chosen
        and they are larger than the CCD image frame
    


In [1]:
%matplotlib widget

from __future__ import division

import sys
import os
import argparse
import numpy as np
import re

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

import sep as sep

from astropy.io import fits
from astropy import time, coordinates as coord, units as u
from astropy.coordinates import SkyCoord, AltAz

import warnings



In [2]:
###############################################################################

def ellipse_plot(x, y, size, a, b, theta, f_colour, e_colour, number):
    """
    Function to plot an ellipse on a CCD image.

    Keyword arguments:
    x           -- x-axis pixel coordinate centre of ellipse
    y           -- y-axis pixel coordinate centre of ellipse
    a           -- semi-major axis of ellipse
    b           -- semi-minor axis of ellipse
    f_colour    -- facecolour of ellipse
    e_colour    -- edge colour of ellipse
    number      -- number of ellipses to plot
    """

    if len(number) != 1:
        for i in number:
            e = Ellipse(xy=(x, y), width=2.*i*size*a, height=2.*i*size*b,
                        angle=theta * 180. / np.pi)
            e.set_facecolor(f_colour)
            e.set_edgecolor(e_colour)
            if i > 1:
                e.set_linestyle('--')
            ax.add_artist(e)
            plt.draw()
    else:
        e = Ellipse(xy=(x, y), width=2.*size*a, height=2.*size*b,
                    angle=theta * 180. / np.pi)
        e.set_facecolor(f_colour)
        e.set_edgecolor(e_colour)
        ax.add_artist(e)
        plt.draw()

In [3]:
def tick_plot(ax, x, y):
    """
    Function to plot major and minor tick marks on CCD image.

    Keyword arguments:
    ax  -- matplot lib axes object
    x   -- x-axis array of values
    y   -- y-axis array of values
    """

    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    ax.set_xticks(np.arange(0, len(x), 50))
    ax.set_yticks(np.arange(0, len(y), 50))
    ax.set_xticks(np.arange(0, len(x), 10), minor=True)
    ax.set_yticks(np.arange(0, len(y), 10), minor=True)



In [4]:
def get_observatory(observatory, **kwargs):
    """
    Function to find and return coordinates (lon, lat, alt) of an observatory.

    Keyword arguments:
    observatory -- name of observatory of observations.

    NOTE: If you want a *precise* location of a telescope, you can define it
    below as an if statement.
    """
    if observatory == "SAAO":
        obs_loc = coord.EarthLocation.from_geodetic(lat=-32.379663,
                                                    lon=20.810129,
                                                    height=1798.)
    else:
        try:
            obs_loc = coord.EarthLocation.of_site(observatory).to_geodetic()
        except:
            print("\n !!! ERROR: that observatory is not a default."
                  " You can include its location manually by modifying"
                  " the source code, or choose one of the following: \n"
                  " {} \n ".format(coord.EarthLocation.get_site_names()[3:]))
            exit()

    return obs_loc


In [5]:
def find_midpoint(instrument, header, iter):
    """
    Function to find the midpoint of an exposure time.

    Keyword arguments:
    instrument  -- name of the instrument, e.g. SHOC
    header      -- header object of FITS file
    iter        -- iterable interger for ith FITS image
    """

    # SHOC
    if instrument == "SHOC":

        trigger = np.str(header['TRIGGER'])  # needed to define exposure time

        # Internal trigger: FRAME timestamp is *END* of the first exposure
        """
        NOTE: that the FITS file comment is WRONG (see SHOC paper)
        """
        if trigger == "Internal":
            exp_time = float(header['Exposure'])     # units of sec
            ACT_time = float(header['ACT'])          # units of sec
            read_time = (ACT_time - exp_time) / 86400.  # units of days
            exp_time = exp_time / 86400.                # convert into days

            # read time_stamp and *SUBTRACT* half exp_time to get midpoint
            t0 = time.Time(header['FRAME'], format='isot', scale='utc',
                           precision=9)
            midpoint = t0.utc.jd - (0.5*exp_time)

        # External Start: FRAME timestamp is *END* of the first exposure
        elif trigger == "External Start":
            exp_time = float(header['Exposure'])    # units of sec
            exp_time = exp_time / 86400.               # convert into days
            read_time = 0.00676 / 86400.               # hard-code: not in FITS

            # read time_stamp and *SUBTRACT* half exp_time to get midpoint
            t0 = time.Time(header['FRAME'], format='isot', scale='utc',
                           precision=9)
            midpoint = t0.utc.jd - (0.5*exp_time)

        # External trigger: FRAME timestamp is *START* of the first exposure
        elif trigger == "External":
            exp_time = float(header['GPS-INT'])     # units of milli-sec
            exp_time = exp_time / 1000. / 86400.       # convert into days
            read_time = 0.                             # no read-time should be inlcuded when triggering on each frame 

            # read time_stamp and *ADD* half exp_time to get midpoint
            t0 = time.Time(header['FRAME'], format='isot', scale='utc',
                           precision=9)
            midpoint = t0.utc.jd + (0.5*exp_time)

        else:
            print("\n     !!! ERROR: 'TRIGGER' keyword not in FITS header")
            print("                Are you sure this is a SHOC FITS file? \n")
            exit()

        # populate JD midpoint time stamp of 'ith' image frame
        increment = ((float(iter) - 2.) * (read_time + exp_time))
        midpoint_JD = (midpoint + increment)
    # Mookodi
    elif instrument == "Mookodi":

        # read integration (exp_time+read_time) from FITS header
        exp_time = float(header['EXPTIME'])  # units of sec
        exp_time = exp_time / 86400.            # units of days

        # Mookodi gives UTC stamps at *START* of exposures so add on half exp time
        t0 = time.Time(header['DATE-OBS'], format='isot', scale='utc',
                           precision=9)
        midpoint_JD = (t0.utc.jd + 0.5*exp_time)

        read_time = 0.  # since time stamp is accurate for each exposure
        trigger = ""        

    else:
        print("\n     !!! ERROR: unknown instrument specified")
        print("                Exiting program \n")
        exit()

    return exp_time, read_time, t0, midpoint_JD, trigger



In [6]:
def find_instrument(instrument, header):
    """
    Function to extract all necessary information from FITS header given the
    specified instrument, e.g. sensitivity and gain (electrons per ADU)

    Keyword arguments:
    instrument  -- name of the instrument, e.g. SHOC
    header      -- header object of FITS file
    """

    # SHOC: coded such that outptamp == "conventional"
    if instrument == "SHOC":

        # read specific keywords from FITS header
        serial_num = np.str(header['SERNO'])        # instrument serial number
        preamp_gain = np.str(header['PREAMP'])      # look-up sensitivity gain
        pixelreadtime = np.str(header['READTIME'])  # pixel readtime
        outptamp = np.str(header['OUTPTAMP'])       # typically: conventional

        # SHOC1 (SHA)
        if serial_num == "5982":
            if pixelreadtime == "3.333333e-07":
                sensitivity = {'1.0': 10.98, '2.4': 4.23, '4.9': 1.82}
                gain = sensitivity[preamp_gain]
            if pixelreadtime == "1e-06":
                sensitivity = {'1.0': 4.06, '2.4': 1.69, '4.9': 0.63}
                gain = sensitivity[preamp_gain]

        # SHOC2 (SHD)
        elif serial_num == "6448":
            if pixelreadtime == "3.333333e-07":
                sensitivity = {'1.0': 9.92, '2.5': 3.96, '5.2': 1.77}
                gain = sensitivity[preamp_gain]
            if pixelreadtime == "1e-06":
                sensitivity = {'1.0': 3.79, '2.5': 1.53, '5.2': 0.68}
                gain = sensitivity[preamp_gain]

        else:
            print("\n     !!! ERROR: unknown 'SERIAL_NUM' in FITS header")
            print("                Are you sure this is a SHOC FITS file? \n")
            exit()
    elif instrument == "Mookodi":
        gain = float(header['GAIN'])    
    else:
        print("\n     !!! ERROR: unknown instrument specified")
        print("                Exiting program \n")
        exit()

    return gain



In [7]:
def find_object_name(instrument, header):
    """
    Function to extract the object name from a FITS header.

    Keyword arguments:
    instrument  -- name of the instrument, e.g. SHOC
    header      -- header object of FITS file

    NOTE: currently coded for SHOC and STE, but other instruments can be
    included as 'if' statements below and throughout the code
    """
    if instrument == "SHOC":
        star_name = np.str(header['OBJECT'])        
    elif instrument == "Mookodi":
        star_name = np.str(header['OBJECT'])    

    else:
        print("\n     !!! ERROR: unknown instrument specified")
        print("                Exiting program \n")
        exit()

    return star_name


In [8]:
def find_object_coord(instrument, header):
    """
    Function to extract coordinates of a target from FITS header.

    Keyword arguments:
    instrument  -- name of the instrument, e.g. SHOC
    header      -- header object of FITS file

    NOTE: currently coded for SHOC and STE, but other instruments can be
    included as 'if' statements below and throughout the code
    """
    if instrument == "SHOC":
        star_ra = str(header['OBJRA'])
        star_dec = str(header['OBJDEC'])
    elif instrument == "Mookodi":
        star_ra = str(header['TARG_RA '])
        star_dec = str(header['TARG_DEC'])    
    else:
        print("\n     !!! ERROR: unknown instrument specified")
        print("                Exiting program \n")
        exit()

    return star_ra, star_dec



In [9]:
def two_strs(v):
    """
    Function to split two strings into separate variables.

    Keyword arguments:
    v -- string to be separated
    """
    values = v.split()
    if len(values) != 2:
        txt = "\n\n !!! ERROR: RA and DEC coordinates need to be provided" \
              " in the form: \n " \
              "           --coord '19:22:43.56 -21:25:27.53' \n"
        raise argparse.ArgumentTypeError(txt)
    values = list(map(str, values))
    return values

def str2bool(v):
    """
    Function to split two bools into separate variables.

    Keyword arguments:
    v -- string to be separated
    """
    if v.lower() in ('yes', 'Y', 'y', 'true', 't', 'T', 'True', 'TRUE', '1'):
        return True
    elif v.lower() in ('no', 'N', 'n', 'false', 'f', 'False', 'FALSE', '0'):
        return False
    else:
        txt = "\n\n !!! ERROR: a boolean value is expected for these" \
              " optional parameters: \n" \
              "            --do_plot True \n" \
              "            --extract True \n"
        raise argparse.ArgumentTypeError(txt)


def str2flt(v):
    """
    Function to convert a string into a float.

    Keyword arguments:
    v -- string to be converted
    """
    if v.replace('.', '', 1).isdigit():
        return float(v)
    else:
        txt = "\n\n !!! ERROR: a float is expected for these optional" \
              " parameters: \n" \
              "            --source_sigma 10 \n" \
              "            --do_clip 5 \n" \
              "            --extinction 0.25 \n"
        raise argparse.ArgumentTypeError(txt)


## Main Code

In [10]:
    # set default plotting parameter
font = {'family': 'sans-serif',
            'weight': 'normal',
            'size': 14}
matplotlib.rc('font', **font)
matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'

observatory = "SAAO"
instrument = "Mookodi"
star_name = 'e9'
image_dir = "./e9sti"    # Directory path of where image file(s) are located
image = "fbMKD"          # starting file name of image file(s)



source_sigma = 20   #The S/N of the image source extraction
do_plot = False  #Plot and save all figures
#start_frame = args.sqtart_frame


# 'changeable' inputs for APERTURE PHOTOMETRY (only for the expert)

# size of plotting ellipses in images: not used for science
size = 4

# sizes used for curve of growth assessment for optimum aperture size
ap_size = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# pixel tolerance for jitter in source extraction from frame-to-frame
pix_limit = 10

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #


# get coordinates (lon, lat, alt) of specified observatory
obs_loc = get_observatory(observatory)

if instrument == "SHOC":
        cube = False     # SHOC provides FITS cubes as output, but we turned off this function 
elif instrument == "Mookodi":
        cube = False  # Mookodi provides individual FITS files as output
        extract = False  # only allow extract if cube not False    
else:
        print("\n !!! ERROR: unknown instrument specified.\n"
            "            available instruments are: \n"
            "            {} \n".format(known_instruments))
        exit()

    
    
# define empty lists for light curves
final_time_HJD = []
final_time_BJD = []
targ_mag = []
targ_mag_err = []
comp_mag = []
comp_mag_err = []
targ_mag_corr = []
comp_mag_corr = []
targ_airmass = []

# define empty lists for curve of growth
cg_area = []
cg_flux = []
cg_sky = []
cg_area_comp = []
cg_flux_comp = []

if not os.path.exists(image_dir):
        print("\n     !!! ERROR: specified IMAGE directory does not"
              " exist: {} \n".format(image_dir))
        exit()
        
image_files = []
for file in os.listdir(image_dir):
          if image in file:
                image_files.append(file)
image_files.sort()

if len(image_files) == 0:
        print("\n     !!! ERROR: no IMAGE files containing the string:"
                "  {} found in {} \n".format(image, image_dir))
        exit()
n_slice = len(image_files)

hdulist = fits.open(image_dir+'/'+image_files[0],
                       ignore_missing_end=True)
hdu = hdulist[0]
header = hdulist[0].header
hdulist.close()
        

print("\n (*) Running aperture photometry script for {} photometry"
          " from {} ...".format(instrument, observatory))
star_ra, star_dec = find_object_coord(instrument, header)



 (*) Running aperture photometry script for Mookodi photometry from SAAO ...


## Loop over all image frames. (might want to disable if the stars move a lot between images)

In [11]:
n_start = 0
edge_mask = np.zeros(raw_image.shape, dtype=bool)
edge_mask[:] = False

for i in range(n_start, n_slice):
            hdulist = fits.open(image_dir+'/'+image_files[i],
                                ignore_missing_end=True)
            hdu = hdulist[0]
            header = hdulist[0].header
            hdu.data = hdu.data

# boolean to do aperture photometry unless error
            do_targ, do_comp = True, True

        # find correct gain sensitivity from instrument look-up table
            gain = find_instrument(instrument, header)

        # find exposure time and image frame midpoint (in JD)
            exp_time, read_time, t0, midpoint_JD, \
                trigger = find_midpoint(instrument, header, int(i))

        # define time of first exposure in cube
            if int(i) == n_start:
                t0_start = t0
            red_image = hdu.data.astype(dtype=float)
            
                    # create astropy time object in JD format
            JD_time = time.Time(midpoint_JD, format='jd', scale='utc',
                            location=obs_loc)
            star_coord = SkyCoord(star_ra, star_dec, frame='icrs',
                              unit=(u.hourangle, u.deg))

        # calculate HJD (UTC) and truncate (-2400000)
            ltt_helio = JD_time.light_travel_time(star_coord, 'heliocentric')
            midpoint_HJD = (JD_time.utc + ltt_helio)
            midpoint_HJD = float(midpoint_HJD.value) - 2400000.

        # calculate BJD (TDB) and truncate
        
        #NOTE: HJD (UTC) and BJD (TDB) can be up to +/- 4 mins different from
        #each other, hence should not be ignored when studying short-period
        #variability

            ltt_bary = JD_time.light_travel_time(star_coord, 'barycentric')
            midpoint_BJD = (JD_time.tdb + ltt_bary)
            midpoint_BJD = float(midpoint_BJD.value) - 2400000.

        # calculate airmass correction using target star's coordinates
            staraltaz = star_coord.transform_to(AltAz(obstime=JD_time,
                                                  location=obs_loc))
            z = 90. - (staraltaz.alt.degree)
            z = np.deg2rad(z)
            secz = 1. / np.cos(z)
            X = secz - 1.8167e-3*(secz-1.) - 2.875e-3*((secz-1.)**2.) \
                - 8.083e-4*((secz-1.)**3.)
            
        # user source selection 
            bkg = sep.Background(red_image, mask=edge_mask)
            red_image = red_image - bkg
            
        # SEP source extraction requiring minimum of 8 pixels for each source
            objects = sep.extract(red_image, source_sigma, err=bkg.globalrms,
                              mask=edge_mask, minarea=8, gain=gain)
        # define min/max location of all extracted sources
            x_min_pix_objects = objects['xmin']
            x_max_pix_objects = objects['xmax']
            y_min_pix_objects = objects['ymin']
            y_max_pix_objects = objects['ymax']
            
            if i == n_start:
                print("\n (*) Plotting first IMAGE frame ...")

            # plot 'first' image
                fig3, ax = plt.subplots(figsize=(6, 6), num=3)
                plt.subplots_adjust(left=0.15, right=0.9, top=0.95, bottom=0.15)
                m, s = np.mean(red_image), np.std(red_image)
                im = ax.imshow(red_image, interpolation='nearest',
                           cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
                tick_plot(ax, red_image[0], red_image[1])  # call tick function
                ax.grid(which='major', color='y', linestyle='-', linewidth=0.5)

            # overplot objects from source extraction
                for j in range(len(objects)):
                    ellipse_plot(objects['x'][j], objects['y'][j], size,
                                 objects['a'][j], objects['b'][j],
                                 objects['theta'][j], 'none', 'red', [1])

                if do_plot:
                    plt.savefig(out_dir+"extracted_sources.pdf", dpi=300)

                plt.show(block=False)
                plt.pause(0.1)
                if (len(objects) >= 1) and (len(objects) <= 2):
                    print("\n     ! WARNING: not many sources have been detected")
                    print("                Perhaps source_sigma is too high? \n")
                elif len(objects) == 0:
                    print("\n !!! ERROR: no sources detected")
                    print("     Try using a smaller source_sigma")
                    print("     Exiting program in 10 seconds")
                    plt.pause(10)
                    exit()

                while True:
                    try:
                        toolbar = plt.get_current_fig_manager().toolbar
                        if toolbar.mode == '':
                            print("\n     >>> Click the image to choose the"
                                  " TARGET star's co-ordinates:")
                        click = fig3.ginput(1, timeout=0, show_clicks=False)
                        init_pix_x = list(map(lambda x: x[0], click))
                        init_pix_y = list(map(lambda x: x[1], click))

                        # find star nearest to user's input
                        index = np.r_[(init_pix_x < x_max_pix_objects+pix_limit)
                                      & (init_pix_x > x_min_pix_objects-pix_limit)
                                      & (init_pix_y < y_max_pix_objects+pix_limit)
                                      & (init_pix_y > y_min_pix_objects-pix_limit)]

                    except ValueError:
                        print("\n     !!! ERROR: you need to click on the"
                              " screen to choose a target star")
                        continue

                    # break loop if target coordinates found
                    if index.any() == True:  # MUST(!) be '==' and not 'is'
                        tmp_x = ("{:.2f}".format(float(objects['x'][index])))
                        tmp_y = ("{:.2f}".format(float(objects['y'][index])))
                        print("     > Found target star located at: \n"
                              "     > (X,Y) = ({}, {}) ".format(tmp_x, tmp_y))
                        break
                    else:
                        toolbar = plt.get_current_fig_manager().toolbar
                        if toolbar.mode == '':
                            print("\n     !!! ERROR: no target object found!")
                            continue
                            
            # overplot target star aperture(s)
                ellipse_plot(objects['x'][index], objects['y'][index], size,
                             objects['a'][index], objects['b'][index],
                             objects['theta'][index], 'none', 'green',
                             [1., 1.5, 2.])

                plt.pause(0.1)

    

NameError: name 'raw_image' is not defined

##  USER SOURCE SELECTION
